In [1]:
import numpy as np
import os
import sys

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision
from torchvision import datasets, models, transforms

from source import functions
from source import cifar_dataloader


print('python version : ', sys.version)
print('torch version : ', torch.__version__)
print('torchvision version : ', torchvision.__version__)
print('device number : ', torch.cuda.device_count())

python version :  3.8.13 (default, Oct 21 2022, 23:50:54) 
[GCC 11.2.0]
torch version :  1.13.0+cu117
torchvision version :  0.14.0+cu117
device number :  4


In [6]:
from warnings import filterwarnings
filterwarnings('ignore')

# nn.DataPararell로 병렬처리

In [18]:
usage = 4
multiple_gpus = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
devices = multiple_gpus[:4]
device_0 = devices[0]
device = torch.device('cuda')

In [21]:
model,input_size = functions.get_model('resnet', pretrained=True, transfer=True, feature_extract=False, num_classes=10)

In [26]:
path = './../data/'
dataloaders_dict = cifar_dataloader.get_cifar10_dataloader(path, input_size, 16, 4)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
lr = 0.0001
num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)

In [34]:
print(f'devices = {devices},\ndevice_0 = {device_0},\ndevice = {device}')

devices = [device(type='cuda', index=0), device(type='cuda', index=1), device(type='cuda', index=2), device(type='cuda', index=3)],
device_0 = cuda:0,
device = cuda


In [37]:
## 단일 gpu 사용
# model = model.to(device_0)

# multi-gpu 사용
model = nn.DataParallel(model, device_ids=devices)
# model = model.cuda()

In [38]:
# model은 nn.DataParallel로 감싸져 있음
# 데이터를 받는 device는 "cuda"
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()  
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders_dict[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

Epoch 0/4
train Loss: 0.9103 Acc: 0.7013
val Loss: 0.3108 Acc: 0.8978
Epoch 1/4
train Loss: 0.8249 Acc: 0.7335
val Loss: 0.2944 Acc: 0.9048
Epoch 2/4
train Loss: 0.7672 Acc: 0.7514
val Loss: 0.2693 Acc: 0.9132
Epoch 3/4
train Loss: 0.7216 Acc: 0.7658
val Loss: 0.2526 Acc: 0.9147
Epoch 4/4
train Loss: 0.6928 Acc: 0.7744
val Loss: 0.2300 Acc: 0.9258
